In [52]:
import pandas as pd
import duckdb

In [53]:
# create an in-memory temporary database by using duckdb.connect()
conn = duckdb.connect()

In [54]:
group1_df = pd.read_csv("./Group_1.csv")

In [55]:
group2_df = pd.read_csv("./Group_2.csv")

In [56]:
# save data-frame to duckdb table 
conn.register("group_1_txt", group1_df)
conn.register("group_2_txt", group2_df)


In [63]:
# execute the query 
# fetchdb() returns a dataframe
result_df =conn.execute("""

select * from group_1_txt

""").fetchdf()

In [64]:
result_df

,PersonID,Height_CM,DOB
0,14521,160,1963-9-3
1,15321,171,1964-2-12
2,21821,175,1976-10-11
3,31721,169,1981-2-12
4,42421,167,1982-3-12
5,11431,163,1979-4-12
6,16123,162,1989-6-10


In [60]:
query1 = """

with Group1_Enrich as (
select 
  PersonID, 
  CAST(Height_CM as decimal) as Height,
  CAST(DOB AS DATE) as BirthDate
from group_1_txt
),

Group2_Enrich as (
select 
  PersonID, 
  CAST(Height_CM as decimal) as Height, 
  CAST(DOB AS DATE) as BirthDate
from group_2_txt
),
Group1_Min_BirthDate as (
SELECT
     BirthDate,
    'Group1' as GroupName,
    PersonID,
    Height,
    'Eldest' as Remarks
from Group1_Enrich
ORDER BY BirthDate ASC
Limit 1
),
Group1_Max_Height as (
select 
   BirthDate,
  'Group1' as GroupName,
   PersonID,
   Height,
  'Tallest' as Remarks
from Group1_Enrich
ORDER BY Height DESC
Limit 1
),

Group2_Min_BirthDate as (
select 
    BirthDate,
   'Group2' as GroupName,
   PersonID,
   Height,
   'Eldest' as Remarks
 from Group2_Enrich
 ORDER BY BirthDate ASC
 LIMIT 1
),

Group2_Max_Height as (
select 
  BirthDate,
  'Group2' as GroupName,
  PersonID,
  Height,
  'Tallest' as Remarks
  from Group2_Enrich
  ORDER BY Height DESC
  Limit 1
)

---- Return the final result 
select 
*
from 
 Group1_Min_BirthDate
union all
select 
*
from 
  Group1_Max_Height
union all
select 
*
from 
  Group2_Min_BirthDate
union all
select 
*
from   
  Group2_Max_Height
  
"""

In [61]:
result1_df =conn.execute(query1).fetchdf()

In [62]:
result1_df

,BirthDate,GroupName,PersonID,Height,Remarks
0,1963-09-03,Group1,14521,160.0,Eldest
1,1993-10-11,Group2,21824,175.0,Tallest
2,1981-02-12,Group2,39721,169.0,Eldest
3,1976-10-11,Group1,21821,175.0,Tallest
